<a href="https://colab.research.google.com/github/Frank-Howd/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/LS_DS_223_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


**You can't just copy** from the lesson notebook to this assignment.

- Because the lesson was **regression**, but the assignment is **classification.**
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [26]:
# COMPLETE IMPORTS, WRANGLE DATA

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from category_encoders import OneHotEncoder, OrdinalEncoder
from pandas_profiling import ProfileReport
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

def wrangle(df):
    """Takes in a dataframe, creates a deep copy, cleans the data, and returns a
    df with the resultant changes to instances and features"""

    # Make a copy, keep the original DataFrame in memory
    df = df.copy()

    # Drop duplicate rows
    df.drop_duplicates(inplace=True)

    # Set index to 'id'
    df.set_index(keys='id', inplace=True)

    # Turn 'date_recorded' into datetime object and fill column with dt.year
    df['date_recorded'] = pd.to_datetime(df['date_recorded'], 
                                         infer_datetime_format=True).dt.year
    
    # Replace 'Null Island' with NaN values
    df.replace([0, -2.000000e-08], np.nan, inplace=True)

    # Feature engineering
    df['pump_age'] = df['date_recorded'] - df['construction_year']

    # Drop duplicate columns
    df.drop(columns=['extraction_type_group', 'quantity_group', 'recorded_by'], inplace=True)

    # Drop high cardinality columns
    drop_cols = [col for col in df.select_dtypes('object').columns if 
                 df[col].nunique() > 100]
    df.drop(columns=drop_cols, inplace=True)

    # Drop features with high NaN counts
    # df.dropna(axis=1, thresh=len(df)*0.8, inplace=True)

    return df

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [27]:
train = wrangle(train)
test = wrangle(test)

In [28]:
print(train.shape, test.shape)
print(train.columns, "\n", test.columns)

(59400, 31) (14358, 30)
Index(['amount_tsh', 'date_recorded', 'gps_height', 'longitude', 'latitude',
       'num_private', 'basin', 'region', 'region_code', 'district_code',
       'population', 'public_meeting', 'scheme_management', 'permit',
       'construction_year', 'extraction_type', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group',
       'status_group', 'pump_age'],
      dtype='object') 
 Index(['amount_tsh', 'date_recorded', 'gps_height', 'longitude', 'latitude',
       'num_private', 'basin', 'region', 'region_code', 'district_code',
       'population', 'public_meeting', 'scheme_management', 'permit',
       'construction_year', 'extraction_type', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity'

In [29]:
# Split data into a Feature Matrix and a Target Vector

target = 'status_group'

y_train = train[target]
X_train = train.drop(columns=target)

print(X_train.shape, y_train.shape)
print(test.shape)

(59400, 30) (59400,)
(14358, 30)


In [30]:
# Establish a Basline Accuracy

print("Baseline accuracy is most frequent group: {:.4f}".format(
    y_train.value_counts(normalize=True).max()))

Baseline accuracy is most frequent group: 0.5431


In [31]:
high_card = [col for col in X_train.select_dtypes('object').columns if X_train[col].nunique() > 10]
low_card = [col for col in X_train.select_dtypes('object').columns if X_train[col].nunique() <= 10]

display(X_train.select_dtypes('object').nunique().sort_values())
print("\n", high_card, end=' ')
print(low_card)

ct = ColumnTransformer([
                        ('ohe', OneHotEncoder(use_cat_names=True), low_card),
                        ('ordinal', OrdinalEncoder(), high_card)
])

public_meeting            1
permit                    1
source_class              3
quantity                  5
management_group          5
waterpoint_type_group     6
quality_group             6
source_type               7
payment_type              7
payment                   7
extraction_type_class     7
waterpoint_type           7
water_quality             8
basin                     9
source                   10
management               12
scheme_management        12
extraction_type          18
region                   21
dtype: int64


 ['region', 'scheme_management', 'extraction_type', 'management'] ['basin', 'public_meeting', 'permit', 'extraction_type_class', 'management_group', 'payment', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group']


In [32]:
# Make a pipeline for a random forest classifier

rf = make_pipeline(
    OrdinalEncoder(),    
    SimpleImputer(),
    RandomForestClassifier(n_estimators=20, random_state=42)
)


In [33]:
# Obtain cross validation scores (KFolds = 5)

rf_cvs = cross_val_score(rf, X_train, y_train, cv=5, n_jobs=-1)

In [34]:
# Print out the scores and a couple of summary statistics

print('Cross validation scores for regression tree model:', rf_cvs)
print('\n Mean cross_val_score:', rf_cvs.mean(), end=' ')
print('\n Standard deviation:', rf_cvs.std())

# Ordinal score 0.786818
# Column Transform score 0.775319

Cross validation scores for regression tree model: [0.8020202  0.79436027 0.80252525 0.79747475 0.79755892]

 Mean cross_val_score: 0.7987878787878787 
 Standard deviation: 0.003074223428207494


In [35]:
rf

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=None, drop_invariant=False,
                                handle_missing='value', handle_unknown='value',
                                mapping=None, return_df=True, verbose=0)),
                ('simpleimputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='mean',
                               verbose=0)),
                ('randomforestclassifier',
                 RandomForestClassifier...p=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                        

In [36]:
# from scipy.stats import randint

# Establish parameters for model tuning

params = {'randomforestclassifier__n_estimators': np.arange(20, 90, 30),
         'randomforestclassifier__max_depth': np.arange(10, 31, 10),
         'randomforestclassifier__max_samples': np.arange(0.7, 1.0, 0.1)}

# params = {"randomforestclassifier_max_depth": [12, None],
#               "randomforestclassifier_max_features": randint(1, 91),
#               "randomforestclassifier_min_samples_split": randint(2, 91),
#               "randomforestclassifier_bootstrap": [True, False],
#               "randomforestclassifier_criterion": ["gini", "entropy"]}

In [37]:
# Complete a Randomized Search for hyperparameter optimization

rf_rs = RandomizedSearchCV(rf, param_distributions=params, n_iter=4, cv=5, 
                           n_jobs=-1, verbose=1)

In [38]:
# Can visualize your pipeline 
# from sklearn import set_config
# set_config(display='diagram')
# rf

In [39]:
# Fit data to optimized model

rf_rs.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  1.4min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            fill_value=None,


In [40]:
# See the best score

rf_rs.best_score_

0.8046464646464647

In [41]:
# Identify the best hyperparameter settings

rf_rs.best_params_

{'randomforestclassifier__max_depth': 30,
 'randomforestclassifier__max_samples': 0.9999999999999999,
 'randomforestclassifier__n_estimators': 80}

In [42]:
# Create Submission

y_pred = rf_rs.predict(test)

In [43]:
submission = pd.DataFrame({'status_group': y_pred}, index=test.index)

In [44]:
submission[:10]

,status_group
id,
50785,functional
51630,functional
17168,functional
45559,non functional
49871,functional
52449,functional
24806,non functional
28965,non functional
36301,non functional


In [45]:
# Export submission to .csv file

submission.to_csv('submission6.csv')

In [48]:
importances = rf_rs.best_estimator_.named_steps['randomforestclassifier'].feature_importances_
features = X_train.columns



In [57]:
pd.Series(importances, index=features).sort_values().tail(10).plot(kind='barh')


In [ ]:
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']
 
target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]
 
sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)